# RAG (Retrieval-Augmented Generation) Hands-on Lab

### RAG 개요

RAG는 언어 모델의 성능을 개선하는 간단하면서도 유용한 기법으로 두 단계 프로세스로 이뤄집니다.

첫 번째 단계로 사용자가 입력한 프롬프트를 임베딩하여 지식 소스에서 관련 문서를 검색하는데, 이는 네이버나 구글 검색에서 관련 검색 결과를 가져오는 방식과 같습니다.
임베딩에 특화된 모델을 사용하거나 대규모 언어 모델을 임베딩 모델로 사용할 수 있죠. 지식 소스는 인메모리 DB로 FAISS를 사용하거나 ChromaDB와 같은 벡터 DB, 아니면 OpenSearch를 적용할 수 있습니다.

두 번째 단계에서는 검색 결과를 같이 프롬프트에 포함하여 LLM에 유입함으로써 최종 응답 결과를 생성합니다. LLM의 답변 범위를 검색 결과로 제한함으로써 모델 환각(hallucination) 현상을 완화합니다.

## Step 1. Prepare Large Language Model (LLM) and Embedding Model 
---

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
import time
import sagemaker, boto3, json
import glob
import os
import pandas as pd
import requests
import json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from ssm import parameter_store
from termcolor import colored
from common import get_apigateway_url

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

RESTAPI_ID, URL = get_apigateway_url()
print("RESTAPI_ID = ", RESTAPI_ID)
print("API GATEWAY URL = ", URL)

RESTAPI_ID =  a6uibyuabj
API GATEWAY URL =  https://a6uibyuabj.execute-api.us-west-2.amazonaws.com/api/


In [3]:
MODEL_NAME = "FALCON-40B"

LLM_INFO = {
    "LLAMA2-7B": f"{URL}llm/llama2_7b",
    "FALCON-40B": f"{URL}llm/falcon_40b",    
    "KULLM-12-8B": f"{URL}llm/kkulm_12_8b",
}

LLM_URL = LLM_INFO[MODEL_NAME]
EMB_URL = f"{URL}/emb/gptj_6b"

HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

if 'falcon_40b' in LLM_URL:
    LLM_RESPONSE_KEY = "generated_text"
else:
    LLM_RESPONSE_KEY = "generation"
    
print (f'MODEL_NAME: {MODEL_NAME}\nLLM_URL: {LLM_URL}')    

MODEL_NAME: FALCON-40B
LLM_URL: https://a6uibyuabj.execute-api.us-west-2.amazonaws.com/api/llm/falcon_40b


In [4]:
PARAMS = {
    "LLAMA2-7B": {
        'max_new_tokens': 128,
        'top_p': 0.9,
        'temperature': 0.1,
        'return_full_text': False
    },   
    "FALCON-40B": {
        "max_new_tokens": 200,
        "max_length": 256,
        "num_return_sequences": 1,
        "top_p": 0.9,
        "do_sample": True,
        "temperature": 0.4,
        "return_full_text": False,
        "include_prompt_in_result": False
    } 
}

<br>

## Step 2. Ask a question to LLM without RAG
---


In [5]:
from lib_en import Llama2ContentHandlerAmazonAPIGateway, FalconContentHandlerAmazonAPIGateway
from langchain.llms import AmazonAPIGateway

llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)
if MODEL_NAME == "FALCON-40B": llm.content_handler = FalconContentHandlerAmazonAPIGateway()
elif MODEL_NAME in ["LLAMA2-7B", "LLAMA2-13B"]: llm.content_handler = Llama2ContentHandlerAmazonAPIGateway()
params = PARAMS[MODEL_NAME]
llm.model_kwargs = params

### Without providing the context
- 컨텍스트 없이 질의응답 수행 (모델 환각 확인) 

In [6]:
question = "Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words."

payload = {
    'inputs': question,
    'parameters': params
}

print(colored(question, 'green'))
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0][LLM_RESPONSE_KEY])

Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words.

Managed Spot Training in Amazon SageMaker can be used with the following instances: 
- Amazon SageMaker p3 instances 
- Amazon SageMaker p2 instances 
- Amazon SageMaker p3dn instances 
- Amazon SageMaker p2dn instances 
- Amazon SageMaker p3.2xlarge instances 
- Amazon SageMaker p3.8xlarge instances 
- Amazon SageMaker p3.16xlarge instances 
- Amazon SageMaker p2.xlarge instances 
- Amazon SageMaker p2.8xlarge instances 
- Amazon SageMaker p2.16xlarge instances 
- Amazon SageMaker p2.xlarge instances with NVIDIA GPUs 
- Amazon SageMaker p2.8xlarge instances with NVIDIA GPUs 
- Amazon SageMaker p2.16xlarge instances with NVIDIA GPUs 
- Amazon SageMaker p3.2xlarge instances with NVIDIA GPUs 
- Amazon SageMaker p3.8xlarge instances with NVIDIA GPUs 
- Amazon SageMaker


### With Context
- 추가 컨텍스트 or few-shot 제공

In [7]:
context = """Managed Spot Training can be used with all instances supported in Amazon SageMaker. 
Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available."""
    
prompt = """Answer based on context:\n\n{context}\n\n{question}"""

text_input = prompt.replace("{context}", context)
text_input = text_input.replace("{question}", question)

payload = {
    'inputs': text_input,
    'parameters': params
}

print(colored(text_input, 'green'))

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0][LLM_RESPONSE_KEY])

Answer based on context:

Managed Spot Training can be used with all instances supported in Amazon SageMaker. 
Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available.

Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words.

As an AI language model, I do not have access to the current availability of instances in Amazon SageMaker. However, Managed Spot Training is supported on all instance types that are currently available in Amazon SageMaker. This includes both GPU and CPU instances.


### Apply LangChain


In [8]:
result = llm(text_input)
print(result)


As an AI language model, I don't have access to the current list of instances supported by Amazon SageMaker. However, based on the official documentation, Managed Spot Training can be used with all instances supported in Amazon SageMaker. It is recommended to check the SageMaker documentation for the latest list of supported instances.


<br>

## Step 3. Use RAG based approach with [LangChain](https://python.langchain.com/en/latest/index.html) 
---

### Document Loaders
<img src="../images/RAG-Page-3.png"/>

Document loader를 사용하여 원본 소스에서 데이터를 문서로 로드합니다. 문서는 텍스트와 관련 메타데이터를 의미합니다. 예를 들어 간단한 텍스트 파일을 로드하거나 웹페이지의 텍스트 콘텐츠를 로드하거나 YouTube 동영상의 스크립트를 로드하기 위한 Document loader가 있습니다. Document loader는  기본적으로 'load' 메서드를 사용하며, 상황에 따라 'lazy load'도 사용할 수 있습니다.

pdf, html, json, txt, csv와 같은 다양한 파일 유형에 사용할 수 있는 다양한 'loader'는 물론 Slack, Twitter 등과 같은 타사 플랫폼과의 통합도 지원합니다. 전체 목록은 여기에서 확인해 주세요. https://python.langchain.com/docs/modules/data_connection/document_loaders

In [9]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA

이제 예제 데이터를 다운로드합니다. Amazon SageMaker FAQ (https://aws.amazon.com/sagemaker/faqs/) 를 지식 라이브러리로 사용하겠습니다. 데이터는 질문과 답변의 두 열이 있는 CSV 파일로 구성되며, 이 중에서 답변 열만 지식 라이브러리의 문서로 사용하여 쿼리 기반으로 관련 문서를 검색합니다.

**필요에 따라 예제 데이터 세트를 여러분의 QnA 데이터 세트로 대체하여 구축할 수 있습니다.**

In [10]:
dataset_folder = "../dataset"
save_dataset_path = f"{dataset_folder}/processed/processed_data.csv"

In [11]:
import os, glob
import pandas as pd
all_files = glob.glob(os.path.join(f"{dataset_folder}/raw/", "*_FAQs*.csv"))

In [13]:
df_knowledge = pd.concat(
    (pd.read_csv(f, header=None, names=["Question", "Answer"]) for f in all_files),
    axis=0,
    ignore_index=True,
)

In [14]:
df_knowledge

,Question,Answer
0,What is Amazon SageMaker?,Amazon SageMaker is a fully managed service to...
1,In which Regions is Amazon SageMaker available...,For a list of the supported Amazon SageMaker A...
2,What is the service availability of Amazon Sag...,Amazon SageMaker is designed for high availabi...
3,How does Amazon SageMaker secure my code?,Amazon SageMaker stores code in ML storage vol...
4,What security measures does Amazon SageMaker h...,Amazon SageMaker ensures that ML model artifac...
...,...,...
149,What are Amazon SageMaker Savings Plans?,Amazon SageMaker Savings Plans offer a flexibl...
150,Why should I use Amazon SageMaker Savings Plans?,If you have a consistent amount of Amazon Sage...
151,How can I get started with Amazon SageMaker Sa...,You can get started with Savings Plans from AW...
152,How are Savings Plans for Amazon SageMaker dif...,The difference between Savings Plans for Amazo...


In [15]:
#drop the question column as we're not using it for the exercise.
df_knowledge.drop(["Question"], axis=1, inplace=True)

#saving the modified df 
df_knowledge.to_csv(save_dataset_path, header=False, index=False)

df_knowledge.head(5)

,Answer
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...


In [16]:
csv_loader = CSVLoader(file_path=save_dataset_path)

In [17]:
documents = csv_loader.load()

for document in documents:
    content = document.page_content
    metadata = document.metadata
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
    chunks = text_splitter.split_documents([document])
    
    print(f"=== content ===\n{content}")
    print(f"=== metadata ===\n{metadata}")
    print(f"=== chunks ===\n{chunks}")
    break

=== content ===
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: For a list of the supported Amazon SageMaker AWS Regions, please visit the AWS Regional Services page. Also, for more information, see Regional endpoints in the AWS general reference guide.
=== metadata ===
{'source': '../dataset/processed/processed_data.csv', 'row': 0}
=== chunks ===
[Document(page_content='Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 0}), Document(page_content='learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: For', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 0}), Document(page_content='a list of the supported Amazon SageMaker AWS Regions, please visit the\xa0AWS Region

In [19]:
from lib_en import EmbeddingAmazonApiGateway
emb = EmbeddingAmazonApiGateway(api_url=EMB_URL)

prompt = "What is Amazon SageMaker's advantages for Data Scientists? Please summarize in 100 words"

result = emb.embed_query(prompt)
print(result[0:5])

emb_results = emb.embed_documents([prompt])
print(emb_results[0][:5])

[0.013422408141195774, -0.003006801474839449, -0.0014634798280894756, -0.013446807861328125, -0.01952643319964409]
[0.013422408141195774, -0.003006801474839449, -0.0014634798280894756, -0.013446807861328125, -0.01952643319964409]


In [25]:
result == emb_results[0]

True

### Create the VectorstoreIndex

RAG는 `VectorstoreIndexCreator` 로 쉽고 빠르게 구현할 수 있습니다. 다만, 프롬프트 커스터마이징 및 세부 파라메터 설정이 필요하거나 보다 세밀한 디버깅 시에는 아래 절 (Step 4.)의 과정을 거치는 것을 권장합니다.
- FAISS: https://github.com/facebookresearch/faiss
- LangChain document: https://python.langchain.com/docs/modules/data_connection/vectorstores/

In [26]:
# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0, separators=[" ", ",", ".", "\n"])
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS, # use FAISS as the vectorestore to index and search embeddings
    embedding=emb,
    text_splitter=text_splitter
)

In [28]:
%%time
index = index_creator.from_loaders([csv_loader])

CPU times: user 871 ms, sys: 37.3 ms, total: 909 ms
Wall time: 43.1 s


In [33]:
answer = index.query(question=question, llm=llm)
print(colored(question, 'green'))
print(answer)

Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words.
 Managed Spot Training in Amazon SageMaker can be used with any instance type that supports Spot instances, including EC2 instances, Amazon EC2 Spot instances, and Amazon EC2 Spot Fleet. Spot instances are available at up to a 90% discount compared to On-Demand instances, making them an affordable option for training machine learning models.


<br>

## Step 4. Customize the QA application above with different prompt
---

### Alternative: Use the vectorstore index as a retriever within a RetrievalQA chain

위의 예시처럼 RAG를 매우 편리하고 빠르게 구현할 수 있지만, `VectorstoreIndex`는 "블랙박스"처럼 사용 중인 프롬프트를 완전히 제어할 수 있는 옵션이 제공되지 않습니다. 

이 경우에는 index를 "retriever(검색기)"로 래핑하고 사용자 지정 프롬프트 템플릿을 활용하는 RetrievalQA 객체와 vectorstore index를 retriever 객체로 사용할 수 있습니다.

#### Vectorstore Retriever Options

In [35]:
retriever = index.vectorstore.as_retriever()
print(retriever.get_relevant_documents(question))

[Document(page_content='that you use, not for individual items. You can read more about this in our metering guide. In addition to the notebooks, you can also start and run terminals and interactive shells in SageMaker Studio, all on the same compute instance. Each application runs within a container or image. SageMaker', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 64}), Document(page_content='your Managed Spot Training jobs from restarting if capacity is pre-empted. When you enable checkpoints, Amazon SageMaker resumes your Managed Spot Training jobs from the last checkpoint.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 87}), Document(page_content='set of solutions for the most common use cases that can be deployed readily with just a few clicks. The solutions are fully customizable and showcase the use of AWS CloudFormation templates and reference architectures so you can accelerate your ML journey. SageMaker JumpStart also support

retriever 래핑 시에는 Similarity Search가 디폴트로 적용되지만, MMR(Max Marginal Relevance)를 적용할 수도 있습니다. search_kwargs argument 또한 선택적으로 입력할 수 있으며 주요 파라메터는 아래와 같습니다.

- `k`: top_k의 문서 개수로 기본값은 4입니다.
- `score_threshold`: "similarity_score_threshold" 검색 유형을 사용하는 경우 검색기가 반환하는 문서의 최소 관련성을 설정할 수 있습니다.
- `fetch_k`: MMR 알고리즘에 전달할 문서 개수로 기본값은 20입니다.
- `lambda_mult`: MMR 알고리즘이 반환하는 결과의 다양성을 제어하며, 1은 최소 다양성, 0은 최대 다양성입니다. 기본값은 0.5입니다.
- `filter`: 문서의 메타데이터를 기반으로 검색할 문서에 대한 필터를 정의할 수 있습니다. 벡터스토어에 메타데이터가 저장되어 있지 않은 경우에는 이 옵션이 적용되지 않습니다.

In [36]:
retriever = index.vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":3, "fetch_k": 10})
print(retriever.get_relevant_documents(question))

[Document(page_content='that you use, not for individual items. You can read more about this in our metering guide. In addition to the notebooks, you can also start and run terminals and interactive shells in SageMaker Studio, all on the same compute instance. Each application runs within a container or image. SageMaker', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 64}), Document(page_content='any separate AWS services you use within SageMaker Pipelines.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 35}), Document(page_content='your Managed Spot Training jobs from restarting if capacity is pre-empted. When you enable checkpoints, Amazon SageMaker resumes your Managed Spot Training jobs from the last checkpoint.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 87})]


#### Customize your own prompt

In [37]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
###
{context}
###
Question: {question}

Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

#### Retriever wrapping w/ Chains

LangChain은 컨텍스트를 요약하기 위한 다양한 LLM 체인을 제공하며 대표적으로 4가지 방법(Stuff, Refine, Map reduce, Map re-rank)을 지원합니다.

- **Stuff**: 가장 기본적인 방법으로 프롬프트에 모든 관련 데이터를 컨텍스트로 포함시켜 LLM에 전달합니다. 가장 간단한 접근 방식으로 청크 크기가 작고 검색 결과가 많지 않을 때 효과적입니다. 하지만 LLM에는 한 번의 호출로 처리할 수 있는 토큰의 최대 개수인 컨텍스트 길이(context length)가 존재합니다. LLM의 컨텍스트 길이보다 긴 텍스트를 처리할 때에는 청크 크기를 줄이거나 MapReduce나 Refine 같은 다른 방법을 사용해야 합니다.
<img src="https://python.langchain.com/assets/images/stuff-818da4c66ee17911bc8861c089316579.jpg"/>
- **Refine**: 청크된 문서 리스트를 순회하면서 이전 문서의 LLM 중간 답변 결과를 LLM 체인에 컨텍스트로 전달하여 LLM 답변을 개선합니다. 
<img src="https://python.langchain.com/assets/images/refine-a70f30dd7ada6fe5e3fcc40dd70de037.jpg"/>
- **Map reduce**: 개별 데이터 청크에 대한 초기 프롬프트의 힘을 활용하여 문서의 특정 섹션만을 기반으로 요약 또는 답변을 생성합니다. (Map) 그 이후초기 출력 결과를 결합하는 별도의 프롬프트를 사용하여 전체 문서에 걸친 포괄적이고 일관된 요약 또는 답변을 생성합니다. (Reduce) 
<img src="https://python.langchain.com/assets/images/map_reduce-c65525a871b62f5cacef431625c4d133.jpg"/>
- **Map re-rank**: Map reduce와 비슷하지만, 답변이 얼마나 확실한지에 대한 점수를 같이 부여합니다. 최종적으로 가장 높은 점수를 받은 답변이 반환됩니다.
<img src="https://python.langchain.com/assets/images/map_rerank-0302b59b690c680ad6099b7bfe6d9fe5.jpg"/>
보다 자세한 내용은 https://python.langchain.com/docs/modules/chains/document/ 을 참조하기 바랍니다.

In [38]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
    verbose=True
)

In [39]:
results = qa(question)
answer = results["result"]
#answer = qa.run(question)
print(colored(question, 'green'))
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words.
 You can use any instance type that is available in the AWS Spot Market, including the latest generation of instances such as the EC2 C6g, M6g, R6g, and T4g instances.


### Another approach: Step-by-step RAG

좀 더 나아가 위의 `VectorstoreIndexCreator`를 분해하여 내부에서 어떤 일이 일어나는지 살펴보겠습니다.

In [40]:
#using the same loader
documents = csv_loader.load()

#looking into the first docs
print(documents[:2])

[Document(page_content='Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: For a list of the supported Amazon SageMaker AWS Regions, please visit the\xa0AWS Regional Services page. Also, for more information, see\xa0Regional endpoints\xa0in the AWS general reference guide.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 0}), Document(page_content='Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Amazon SageMaker is designed for high availability. There are no maintenance windows or scheduled downtimes. SageMaker APIs run in Amazon’s proven, high-availability data centers, with service stack replication configured across three facilities in each AWS Region to provide fault tolerance

`RecursiveCharacterTextSplitter`로 각 문서를 청킹하고 청킹한 문서를 `.from_documents`로 임베딩한 다음, 임베딩 결과를 벡터 저장소에 저장하고 관련 문서를 색인합니다. 본 예시에서는 FAISS를 사용하지만, 유스케이스에 따라 ChromaDB, OpenSearch 등의 다양한 벡터 저장소 라이브러리나 서비스를 사용할 수 있습니다.

In [41]:
%%time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=5)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# generate embeddings and load that into FAISS
vectorstore = FAISS.from_documents(texts, emb)

CPU times: user 830 ms, sys: 42.7 ms, total: 872 ms
Wall time: 41.7 s


사용자 쿼리를 기반으로 가장 관련성이 높은 상위 k개의 문서를 식별합니다. (예: k = 3) LLM의 토큰 길이가 제안되어 있기에, 상위 k개의 문서만 LLM에 전달함으로써 컨텍스트 길이를 제어해야 합니다. 

In [42]:
docs = vectorstore.similarity_search(question, k=3)
print(colored(question, 'green'))
print(docs)

Which instances can I use with Managed Spot Training in Amazon SageMaker? Please provide answer within 50 words.
[Document(page_content='that you use, not for individual items. You can read more about this in our metering guide. In addition to the notebooks, you can also start and run terminals and interactive shells in SageMaker Studio, all on the same compute instance. Each application runs within a container or image. SageMaker', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 64}), Document(page_content='your Managed Spot Training jobs from restarting if capacity is pre-empted. When you enable checkpoints, Amazon SageMaker resumes your Managed Spot Training jobs from the last checkpoint.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 87}), Document(page_content='and endpoints, based on\xa0SageMaker pricing. There is no additional charge for using SageMaker JumpStart.', metadata={'source': '../dataset/processed/processed_data.csv', 'row

반드시 retriever로 VectorDB를 사용할 필요가 없으며, 다른 retriever를 사용할 수 있습니다. 아래 코드 스니펫을 참조해 주세요.
- https://python.langchain.com/docs/modules/data_connection/retrievers/

In [43]:
from langchain.retrievers import SVMRetriever

svm_retriever = SVMRetriever.from_documents(texts, emb)
docs_svm = svm_retriever.get_relevant_documents(question)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


마지막으로 검색된 문서를 프롬프트 및 질문과 결합하여 LLM에 입력하여 추론을 수행합니다. 모델 환각 현상이 개선되는 것을 확인하기 바랍니다.

In [44]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
###
{context}
###
Question: {question}

Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT)

In [47]:
result = chain({"input_documents": docs, "question": question})
print(colored(result['input_documents'], 'green'))
print(result["output_text"])

[Document(page_content='that you use, not for individual items. You can read more about this in our metering guide. In addition to the notebooks, you can also start and run terminals and interactive shells in SageMaker Studio, all on the same compute instance. Each application runs within a container or image. SageMaker', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 64}), Document(page_content='your Managed Spot Training jobs from restarting if capacity is pre-empted. When you enable checkpoints, Amazon SageMaker resumes your Managed Spot Training jobs from the last checkpoint.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 87}), Document(page_content='and endpoints, based on\xa0SageMaker pricing. There is no additional charge for using SageMaker JumpStart.', metadata={'source': '../dataset/processed/processed_data.csv', 'row': 28})]
 Managed Spot Training in Amazon SageMaker can be used with the following instances: 
- ml.t2.medium
- m

### ConversationalRetrievalChain

질의응답/채팅 히스토리를 피드백으로 저장하고 그 피드백을 기반으로 이후 대화를 이어나가게 수행이 가능합니다.

In [48]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#retriever = index.vectorstore.as_retriever()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever, verbose=True, memory=memory)

In [49]:
result = qa({"question": "What is SageMaker Ground Truth?"})
print(result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

SageMaker Data Wrangler, Amazon SageMaker Feature Store, Amazon SageMaker Experiments, Amazon SageMaker Debugger, and Amazon SageMaker Model Monitor, can be added to SageMaker Pipelines.

SageMaker Neo documentation.

get started with SageMaker Inference Recommender in minutes while selecting an instance type and get recommendations for optimal endpoint configurations within hours, eliminating weeks of manual testing and tuning time. With SageMaker Inference Recommender, you pay only for the SageMaker ML

Studio notebooks, SageMaker On-Demand notebooks, SageMaker Processing, SageMaker Data Wrangler, SageMaker Training, SageMaker Real-Time Inference, and SageMaker Batch Transform regardless of instance family, size, or Re

In [50]:
result = qa({"question": "What is SageMaker Distributed Training?"})
print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is SageMaker Ground Truth?
Assistant:  SageMaker Ground Truth is a fully managed service that helps you build high-quality labeled datasets for machine learning. It provides a simple and intuitive web-based interface to create labeling jobs, manage human labelers, and monitor their work.
Follow Up Input: What is SageMaker Distributed Training?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

SageMaker Data Wrangler, Amazon SageMaker Feature Store, Amazon SageMaker Experiments, Amazon SageMaker Debugg

In [51]:
result = qa({"question": "What are the two main types of SageMaker Distributed Training??"})
print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is SageMaker Ground Truth?
Assistant:  SageMaker Ground Truth is a fully managed service that helps you build high-quality labeled datasets for machine learning. It provides a simple and intuitive web-based interface to create labeling jobs, manage human labelers, and monitor their work.
Human: What is SageMaker Distributed Training?
Assistant:  SageMaker Ground Truth is a fully managed service that helps you build high-quality labeled data for machine learning (ML) and natural language processing (NLP) applications. SageMaker Distributed Training is a service that allows you to train large-scale deep learning models on Amazon SageMaker.
Follow Up Input: What are the two main types of SageMaker Distributed Training??
Standalone question:

> Finished chain.

<br>

## Step 5. Additional exercises
---

- https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [52]:
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import PDFPlumberLoader

### Wikipedia as source

위키피디아에서 자료 가져오기

In [53]:
wikipedia_loader = WikipediaLoader(query="AWS", load_max_docs=2)
wikipedia_texts = wikipedia_loader.load_and_split(text_splitter=text_splitter)
wikipedia_texts[0]

Document(page_content='Amazon Web Services, Inc. (AWS) is a subsidiary of Amazon that provides on-demand cloud computing platforms and APIs to individuals, companies, and governments, on a metered, pay-as-you-go basis. Clients will often use this in combination with autoscaling (a process that allows a client to use more', metadata={'title': 'Amazon Web Services', 'summary': 'Amazon Web Services, Inc. (AWS) is a subsidiary of Amazon that provides on-demand cloud computing platforms and APIs to individuals, companies, and governments, on a metered, pay-as-you-go basis. Clients will often use this in combination with autoscaling (a process that allows a client to use more computing in times of high application usage, and then scale down to reduce costs when there is less traffic). These cloud computing web services provide various services related to networking, compute, storage, middleware, IoT and other processing capacity, as well as software tools via AWS server farms.  This frees cl

### URLs as source

인터넷 웹페이지 크롤링 - Amazon Rekognition 온라인 문서 추출

In [54]:
urls = [
    "https://docs.aws.amazon.com/rekognition/latest/dg/labels.html", 
    "https://docs.aws.amazon.com/rekognition/latest/dg/faces.html",
    "https://docs.aws.amazon.com/rekognition/latest/dg/collections.html",
    "https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html"
]
url_loader = UnstructuredURLLoader(urls=urls)
url_texts = url_loader.load_and_split(text_splitter=text_splitter)
print(f"Number of splitted texts: {len(url_texts)}")
print(url_texts[0])

Number of splitted texts: 124
page_content='AWS\n\nDocumentation\n\nAmazon Rekognition\n\nDeveloper Guide\n\nLabel Response Objects\n\nDetecting objects and concepts\n\nThis section provides information for detecting labels in images and videos with Amazon Rekognition Image\n        and Amazon Rekognition Video.' metadata={'source': 'https://docs.aws.amazon.com/rekognition/latest/dg/labels.html'}


### PDF source

PDF 소스 활용 - RAG 논문 

In [55]:
import requests
external_dataset_folder = f"{dataset_folder}/external"
os.makedirs(external_dataset_folder, exist_ok=True)

sagemaker_pdf_url = "https://arxiv.org/pdf/2005.11401"
response = requests.get(sagemaker_pdf_url)
file = open(f"{external_dataset_folder}/rag_paper.pdf", "wb")
file.write(response.content)
file.close()

In [56]:
#possible free options: PyPDFLoader, PDFPlumberLoader, PyMuPDFLoader, PDFMinerLoader, PyPDFium2Loader
pdf_loader = PDFPlumberLoader(f"{external_dataset_folder}/rag_paper.pdf")
pdf_texts = pdf_loader.load_and_split(text_splitter=text_splitter)

### Build vector index

위키피디아 + PDF + 웹크롤링 정보로 벡터 인덱스 구축

In [57]:
all_texts = wikipedia_texts + pdf_texts + url_texts
print(f"Number of total texts: {len(all_texts)}")

Number of total texts: 406


In [58]:
%%time
# Embed your texts
agg_vectorstore = FAISS.from_documents(all_texts, emb)

CPU times: user 840 ms, sys: 51.3 ms, total: 891 ms
Wall time: 47.3 s


In [59]:
rekognition_question = "What kind of information does Amazon Rekognition Image returns about image quality?"
aws_question = "What is AWS market share for cloud infrastructure?"
rag_question = "What datasets were used for experiments with RAG?"
questions_list = [rekognition_question, aws_question, rag_question]

In [60]:
for q in questions_list:
    res_docs = agg_vectorstore.similarity_search(q, k=5)
    result = chain({"input_documents": res_docs, "question": q})
    print(colored(q, 'green'))
    print(result["output_text"])
    print("\n")

What kind of information does Amazon Rekognition Image returns about image quality?
 Amazon Rekognition Image does not provide information about image quality. It only detects objects and labels in images.


What is AWS market share for cloud infrastructure?
 As of 2021 Q4, AWS has 33% market share for cloud infrastructure while the next two competitors Microsoft Azure and Google Cloud have 21%, and 10% respectively.


What datasets were used for experiments with RAG?
 The experiments with RAG were conducted using the FEVER dataset, which is a collection of 10,000 claims extracted from Wikipedia and annotated with their verifiability status.


